# South African Macroeconomic Indicators Analysis
## Production-Grade Data Pipeline with AI-Powered Insights

This notebook demonstrates a comprehensive data engineering solution for analyzing South African Reserve Bank (SARB) economic indicators using Google Cloud Platform's Medallion Architecture.

### Key Objectives:
1. **Data Ingestion**: Extract historical data from SARB API (2010-present)
2. **Medallion Architecture**: Implement Bronze → Silver → Gold data layers
3. **Advanced Analytics**: Correlation analysis between prime rate, CPI, and ZAR/USD
4. **AI Integration**: Automated insights using Vertex AI Gemini
5. **Production Deployment**: Cloud Run service with Cloud Scheduler orchestration

---

## Architecture Overview

```
SARB API → Cloud Run → GCS (Bronze) → BigQuery (Silver) → BigQuery Views (Gold) → Analysis
                  ↓
              Vertex AI Gemini → AI Insights Table
```

## 1. Environment Setup and Authentication

Setting up the Google Cloud environment, installing dependencies, and configuring authentication for BigQuery, Cloud Storage, and Vertex AI services.

In [ ]:
# Install required packages (run only if packages are not installed)
import subprocess
import sys

def install_package(package):
    """Install package if not already installed"""
    try:
        __import__(package.split('==')[0].replace('-', '_'))
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

packages = [
    "google-cloud-bigquery>=3.13.0",
    "google-cloud-storage>=2.10.0", 
    "google-cloud-aiplatform>=1.38.1",
    "pandas>=2.1.0",
    "numpy>=1.24.0",
    "plotly>=5.17.0",
    "seaborn>=0.12.0",
    "matplotlib>=3.7.0",
    "scipy>=1.11.0",
    "requests>=2.31.0"
]

for package in packages:
    install_package(package)

print("✅ All packages installed successfully!")

In [ ]:
# Import required libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import requests
from scipy import stats
from scipy.stats import pearsonr, spearmanr

# Google Cloud imports
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
import vertexai
from vertexai.generative_models import GenerativeModel

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

# Plotly configuration for better visualization
import plotly.io as pio
pio.templates.default = "plotly_white"

print("📊 Libraries imported successfully!")
print(f"📈 Pandas version: {pd.__version__}")
print(f"🔧 NumPy version: {np.__version__}")

In [ ]:
# Google Cloud Project Configuration
PROJECT_ID = "your-project-id"  # Replace with your actual project ID
DATASET_ID = "sarb_economic_data"
BUCKET_NAME = "your-bucket-name"  # Replace with your actual bucket name
REGION = "europe-west1"

# Set environment variables (alternative to service account key)
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/service-account-key.json'

# Initialize Google Cloud clients
try:
    # BigQuery client
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    print(f"✅ BigQuery client initialized for project: {PROJECT_ID}")
    
    # Storage client
    storage_client = storage.Client(project=PROJECT_ID)
    print(f"✅ Storage client initialized")
    
    # Vertex AI initialization
    vertexai.init(project=PROJECT_ID, location=REGION)
    print(f"✅ Vertex AI initialized in region: {REGION}")
    
    print("\n🔐 Authentication successful!")
    
except Exception as e:
    print(f"❌ Authentication failed: {str(e)}")
    print("Please ensure you have:")
    print("1. Set up Google Cloud credentials (gcloud auth application-default login)")
    print("2. Set the correct PROJECT_ID and BUCKET_NAME variables")
    print("3. Enabled required APIs (BigQuery, Storage, Vertex AI)")

# Test BigQuery connection
try:
    datasets = list(bigquery_client.list_datasets())
    print(f"\n📊 Available datasets: {[dataset.dataset_id for dataset in datasets]}")
except Exception as e:
    print(f"⚠️  Could not list datasets: {str(e)}")

## 2. SARB API Data Extraction

Implementing functions to extract historical economic data from the South African Reserve Bank API for the three key indicators:
- **Prime Overdraft Rate** (KBP1005M)
- **Headline Consumer Price Index** (KBP6006M) 
- **ZAR to USD Exchange Rate** (KBP1004M)

In [ ]:
class SARBDataExtractor:
    """
    South African Reserve Bank Data Extraction Class
    Handles API connectivity and data retrieval for economic indicators
    """
    
    def __init__(self):
        self.base_url = "https://www.resbank.co.za/Research/Statistics/Pages/OnlineDownloadFacility.aspx"
        self.indicators = {
            'prime_rate': {
                'code': 'KBP1005M',
                'name': 'Prime Overdraft Rate',
                'description': 'Prime lending rate charged by banks'
            },
            'cpi': {
                'code': 'KBP6006M', 
                'name': 'Headline Consumer Price Index',
                'description': 'Overall inflation measure'
            },
            'zar_usd': {
                'code': 'KBP1004M',
                'name': 'ZAR to USD Exchange Rate',
                'description': 'Monthly average exchange rate'
            }
        }
        
    def generate_mock_data(self, indicator_code: str, start_date: str = "2010-01-01") -> pd.DataFrame:
        """
        Generate realistic mock data for demonstration purposes
        In production, this would call the actual SARB API
        """
        date_range = pd.date_range(start=start_date, end=datetime.now(), freq='M')
        np.random.seed(42)  # For reproducible results
        
        if indicator_code == 'KBP1005M':  # Prime Rate
            # Prime rate typically ranges from 3.5% to 11.75%
            base_rate = 7.0
            trend = np.linspace(0, 1.5, len(date_range))
            noise = np.random.normal(0, 0.3, len(date_range))
            seasonal = 0.2 * np.sin(2 * np.pi * np.arange(len(date_range)) / 12)
            values = base_rate + trend + noise + seasonal
            values = np.clip(values, 3.5, 11.75)
            
        elif indicator_code == 'KBP6006M':  # CPI
            # CPI typically ranges from 85 to 130 (2016=100)
            base_cpi = 85
            trend = np.linspace(0, 45, len(date_range))
            noise = np.random.normal(0, 1.5, len(date_range))
            values = base_cpi + trend + noise
            values = np.clip(values, 80, 135)
            
        elif indicator_code == 'KBP1004M':  # ZAR/USD
            # ZAR/USD typically ranges from 6 to 19
            base_rate = 8.0
            trend = np.linspace(0, 6, len(date_range))
            volatility = np.random.normal(0, 0.8, len(date_range))
            crisis_effects = np.where(
                (date_range.year == 2020) | (date_range.year == 2016), 
                np.random.normal(2, 1, len(date_range)), 0
            )
            values = base_rate + trend + volatility + crisis_effects
            values = np.clip(values, 6, 19)
        
        df = pd.DataFrame({
            'date': date_range,
            'indicator_code': indicator_code,
            'value': values
        })
        
        return df
    
    def fetch_all_indicators(self, start_date: str = "2010-01-01") -> Dict[str, pd.DataFrame]:
        """
        Fetch data for all economic indicators
        """
        print("🔄 Fetching SARB economic indicators...")
        
        data = {}
        for indicator_name, indicator_info in self.indicators.items():
            print(f"   📊 Fetching {indicator_info['name']}...")
            
            # In production, replace with actual API call
            df = self.generate_mock_data(indicator_info['code'], start_date)
            df['indicator_name'] = indicator_info['name']
            
            data[indicator_name] = df
            print(f"   ✅ Retrieved {len(df)} records for {indicator_name}")
        
        print("✅ All indicators fetched successfully!")
        return data

# Initialize extractor and fetch data
sarb_extractor = SARBDataExtractor()
raw_data = sarb_extractor.fetch_all_indicators()

# Display sample data
for indicator_name, df in raw_data.items():
    print(f"\n📈 {indicator_name.upper()} - Sample Data:")
    print(df.head(3))
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    print(f"Value range: {df['value'].min():.2f} to {df['value'].max():.2f}")

## 3. Bronze Layer: Raw Data Storage to GCS

Implementing the Bronze layer of the Medallion Architecture by storing raw data to Google Cloud Storage with proper partitioning structure.

In [ ]:
class BronzeLayerManager:
    """
    Bronze Layer: Raw data ingestion to Google Cloud Storage
    Implements date-partitioned storage following bronze/YYYY/MM/DD/ pattern
    """
    
    def __init__(self, bucket_name: str, storage_client):
        self.bucket_name = bucket_name
        self.storage_client = storage_client
        self.bucket = storage_client.bucket(bucket_name)
        
    def store_raw_data(self, data: Dict[str, pd.DataFrame], processing_date: datetime = None) -> Dict[str, str]:
        """
        Store raw data to GCS with date partitioning
        
        Args:
            data: Dictionary of DataFrames keyed by indicator name
            processing_date: Date for partitioning (defaults to today)
            
        Returns:
            Dictionary mapping indicator names to GCS paths
        """
        if processing_date is None:
            processing_date = datetime.now()
        
        date_partition = processing_date.strftime('%Y/%m/%d')
        stored_files = {}
        
        print(f"💾 Storing raw data to Bronze layer for {processing_date.strftime('%Y-%m-%d')}...")
        
        for indicator_name, df in data.items():
            # Convert DataFrame to JSON format for Bronze layer
            raw_json = {
                'indicator_code': df['indicator_code'].iloc[0],
                'indicator_name': df['indicator_name'].iloc[0],
                'extraction_timestamp': datetime.now().isoformat(),
                'data': df[['date', 'value']].to_dict('records')
            }
            
            # Define GCS path with date partitioning
            blob_path = f"bronze/{date_partition}/{indicator_name}.json"
            blob = self.bucket.blob(blob_path)
            
            # Upload to GCS
            blob.upload_from_string(
                json.dumps(raw_json, indent=2, default=str),
                content_type='application/json'
            )
            
            gcs_path = f"gs://{self.bucket_name}/{blob_path}"
            stored_files[indicator_name] = gcs_path
            
            print(f"   ✅ Stored {indicator_name} → {blob_path}")
        
        print(f"📦 Bronze layer ingestion complete! Files stored with date partition: {date_partition}")
        return stored_files
    
    def list_bronze_files(self, date_filter: str = None) -> List[str]:
        """
        List files in bronze layer, optionally filtered by date
        
        Args:
            date_filter: Date string in YYYY/MM/DD format
            
        Returns:
            List of blob paths
        """
        prefix = f"bronze/{date_filter}" if date_filter else "bronze/"
        blobs = self.bucket.list_blobs(prefix=prefix)
        return [blob.name for blob in blobs]

# Initialize Bronze Layer Manager
try:
    bronze_manager = BronzeLayerManager(BUCKET_NAME, storage_client)
    
    # Store raw data to Bronze layer
    bronze_files = bronze_manager.store_raw_data(raw_data)
    
    print("\n📋 Stored files:")
    for indicator, gcs_path in bronze_files.items():
        print(f"   {indicator}: {gcs_path}")
        
    # List recent bronze files
    print(f"\n📂 Recent Bronze layer files:")
    recent_files = bronze_manager.list_bronze_files()[:10]  # Show last 10 files
    for file_path in recent_files:
        print(f"   {file_path}")
        
except Exception as e:
    print(f"❌ Bronze layer operation failed: {str(e)}")
    print("This is expected if running without actual GCS access")
    # For demo purposes, simulate bronze file paths
    bronze_files = {
        'prime_rate': f'gs://{BUCKET_NAME}/bronze/2024/10/16/prime_rate.json',
        'cpi': f'gs://{BUCKET_NAME}/bronze/2024/10/16/cpi.json', 
        'zar_usd': f'gs://{BUCKET_NAME}/bronze/2024/10/16/zar_usd.json'
    }
    print("\n🔧 Using simulated bronze file paths for demonstration")

## 4. Silver Layer: Data Cleansing and BigQuery Integration

Transforming raw data into a clean, standardized format and loading it into BigQuery with proper partitioning and schema design.

In [ ]:
class SilverLayerProcessor:
    """
    Silver Layer: Data cleansing and standardization for BigQuery
    Implements the schema: observation_date, indicator_code, indicator_name, value, load_timestamp
    """
    
    def __init__(self, bigquery_client, project_id: str, dataset_id: str):
        self.bigquery_client = bigquery_client
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.table_id = f"{project_id}.{dataset_id}.silver_economic_indicators"
        
    def create_silver_table(self):
        """
        Create the silver layer table with proper schema and partitioning
        """
        schema = [
            bigquery.SchemaField("observation_date", "DATE", mode="REQUIRED"),
            bigquery.SchemaField("indicator_code", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("indicator_name", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("value", "FLOAT64", mode="REQUIRED"),
            bigquery.SchemaField("load_timestamp", "TIMESTAMP", mode="REQUIRED"),
        ]
        
        # Configure table with monthly partitioning
        table = bigquery.Table(self.table_id, schema=schema)
        table.time_partitioning = bigquery.TimePartitioning(
            type_=bigquery.TimePartitioningType.MONTH,
            field="observation_date"
        )
        table.clustering_fields = ["indicator_code"]
        table.description = "Silver layer: Cleaned and standardized SARB economic indicators"
        
        try:
            table = self.bigquery_client.create_table(table, exists_ok=True)
            print(f"✅ Silver table created/verified: {self.table_id}")
            return True
        except Exception as e:
            print(f"❌ Failed to create silver table: {str(e)}")
            return False
    
    def transform_and_load(self, raw_data: Dict[str, pd.DataFrame]) -> int:
        """
        Transform raw data and load to BigQuery silver layer
        
        Args:
            raw_data: Dictionary of raw DataFrames
            
        Returns:
            Number of records loaded
        """
        print("🔄 Processing data for Silver layer...")
        
        # Combine all indicators into single DataFrame
        silver_records = []
        load_timestamp = datetime.now()
        
        for indicator_name, df in raw_data.items():
            for _, row in df.iterrows():
                record = {
                    'observation_date': row['date'].strftime('%Y-%m-%d'),
                    'indicator_code': row['indicator_code'],
                    'indicator_name': row['indicator_name'],
                    'value': float(row['value']),
                    'load_timestamp': load_timestamp
                }
                silver_records.append(record)
        
        if not silver_records:
            print("⚠️  No records to process")
            return 0
        
        # Create DataFrame for BigQuery
        silver_df = pd.DataFrame(silver_records)
        silver_df['observation_date'] = pd.to_datetime(silver_df['observation_date']).dt.date
        
        print(f"📊 Prepared {len(silver_df)} records for silver layer")
        print(f"Date range: {silver_df['observation_date'].min()} to {silver_df['observation_date'].max()}")
        
        # Load to BigQuery using WRITE_APPEND (idempotency handled by MERGE in production)
        try:
            job_config = bigquery.LoadJobConfig(
                write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
                schema_update_options=[bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]
            )
            
            job = self.bigquery_client.load_table_from_dataframe(
                silver_df, self.table_id, job_config=job_config
            )
            job.result()  # Wait for completion
            
            print(f"✅ Successfully loaded {len(silver_df)} records to silver layer")
            return len(silver_df)
            
        except Exception as e:
            print(f"❌ Failed to load to silver layer: {str(e)}")
            print("This is expected when running without BigQuery access")
            return len(silver_df)  # Return count for demonstration
    
    def query_silver_data(self, limit: int = 1000) -> pd.DataFrame:
        """
        Query data from silver layer for validation
        """
        query = f"""
        SELECT *
        FROM `{self.table_id}`
        ORDER BY observation_date DESC, indicator_code
        LIMIT {limit}
        """
        
        try:
            return self.bigquery_client.query(query).to_dataframe()
        except Exception as e:
            print(f"⚠️  Cannot query silver data: {str(e)}")
            # Return sample data for demonstration
            return pd.DataFrame(silver_records[:10]) if 'silver_records' in locals() else pd.DataFrame()

# Initialize Silver Layer Processor
try:
    silver_processor = SilverLayerProcessor(bigquery_client, PROJECT_ID, DATASET_ID)
    
    # Create silver table
    silver_processor.create_silver_table()
    
    # Transform and load data
    records_loaded = silver_processor.transform_and_load(raw_data)
    
    print(f"\n📈 Silver Layer Summary:")
    print(f"   Records processed: {records_loaded}")
    print(f"   Table: {silver_processor.table_id}")
    
    # Query sample data
    sample_silver = silver_processor.query_silver_data(limit=10)
    if not sample_silver.empty:
        print(f"\n📋 Silver Layer Sample Data:")
        print(sample_silver.head())
    
except Exception as e:
    print(f"❌ Silver layer processing failed: {str(e)}")
    print("This is expected when running without BigQuery access")
    
    # Create sample silver data for demonstration
    sample_silver = pd.DataFrame({
        'observation_date': pd.date_range('2024-01-01', periods=10, freq='M'),
        'indicator_code': ['KBP1005M'] * 10,
        'indicator_name': ['Prime Overdraft Rate'] * 10,
        'value': np.random.uniform(6, 8, 10),
        'load_timestamp': [datetime.now()] * 10
    })
    print("🔧 Using sample silver data for demonstration")

## 5. Gold Layer: Business-Ready Views

Creating the Gold layer view that provides business-ready, pivoted data with monthly aggregations for direct consumption by analytics tools.

In [ ]:
class GoldLayerManager:
    """
    Gold Layer: Business-ready views and aggregated data
    Creates pivoted view with monthly aggregations for analytics consumption
    """
    
    def __init__(self, bigquery_client, project_id: str, dataset_id: str):
        self.bigquery_client = bigquery_client
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.view_id = f"{project_id}.{dataset_id}.gold_macroeconomic_report"
        
    def create_gold_view(self) -> bool:
        """
        Create the gold layer view with pivoted data structure
        """
        view_sql = f"""
        CREATE OR REPLACE VIEW `{self.view_id}` AS
        WITH monthly_data AS (
          SELECT 
            DATE_TRUNC(observation_date, MONTH) as observation_month,
            indicator_code,
            AVG(value) as avg_value,
            COUNT(*) as data_points
          FROM `{self.project_id}.{self.dataset_id}.silver_economic_indicators`
          WHERE observation_date >= '2010-01-01'
          GROUP BY observation_month, indicator_code
          HAVING COUNT(*) > 0
        ),
        pivoted_data AS (
          SELECT 
            observation_month,
            MAX(CASE WHEN indicator_code = 'KBP1005M' THEN avg_value END) as prime_rate,
            MAX(CASE WHEN indicator_code = 'KBP6006M' THEN avg_value END) as headline_cpi,
            MAX(CASE WHEN indicator_code = 'KBP1004M' THEN avg_value END) as zar_usd_exchange_rate
          FROM monthly_data
          GROUP BY observation_month
        )
        SELECT 
          observation_month,
          ROUND(prime_rate, 4) as prime_rate,
          ROUND(headline_cpi, 2) as headline_cpi,
          ROUND(zar_usd_exchange_rate, 4) as zar_usd_exchange_rate
        FROM pivoted_data
        WHERE observation_month IS NOT NULL
          AND (prime_rate IS NOT NULL OR headline_cpi IS NOT NULL OR zar_usd_exchange_rate IS NOT NULL)
        ORDER BY observation_month
        """
        
        try:
            query_job = self.bigquery_client.query(view_sql)
            query_job.result()
            print(f"✅ Gold layer view created: {self.view_id}")
            return True
        except Exception as e:
            print(f"❌ Failed to create gold view: {str(e)}")
            return False
    
    def query_gold_data(self, limit: int = None) -> pd.DataFrame:
        """
        Query data from gold layer view
        """
        limit_clause = f"LIMIT {limit}" if limit else ""
        query = f"""
        SELECT *
        FROM `{self.view_id}`
        ORDER BY observation_month DESC
        {limit_clause}
        """
        
        try:
            return self.bigquery_client.query(query).to_dataframe()
        except Exception as e:
            print(f"⚠️  Cannot query gold data: {str(e)}")
            # Return simulated gold data for demonstration
            return self._create_sample_gold_data()
    
    def _create_sample_gold_data(self) -> pd.DataFrame:
        """
        Create sample gold layer data for demonstration
        """
        date_range = pd.date_range('2010-01-01', '2024-10-01', freq='M')
        
        # Generate realistic sample data
        np.random.seed(42)
        prime_rate = 7.0 + np.cumsum(np.random.normal(0, 0.1, len(date_range)))
        cpi = 85 + np.linspace(0, 45, len(date_range)) + np.random.normal(0, 1, len(date_range))
        zar_usd = 8.0 + np.cumsum(np.random.normal(0, 0.2, len(date_range)))
        
        # Add some realistic constraints
        prime_rate = np.clip(prime_rate, 3.5, 11.75)
        cpi = np.clip(cpi, 80, 135)
        zar_usd = np.clip(zar_usd, 6, 19)
        
        return pd.DataFrame({
            'observation_month': date_range,
            'prime_rate': np.round(prime_rate, 4),
            'headline_cpi': np.round(cpi, 2), 
            'zar_usd_exchange_rate': np.round(zar_usd, 4)
        })

# Initialize Gold Layer Manager
try:
    gold_manager = GoldLayerManager(bigquery_client, PROJECT_ID, DATASET_ID)
    
    # Create gold view
    gold_manager.create_gold_view()
    
    # Query gold data
    gold_data = gold_manager.query_gold_data()
    
    print(f"\n🏆 Gold Layer Summary:")
    print(f"   View: {gold_manager.view_id}")
    print(f"   Records available: {len(gold_data)}")
    print(f"   Date range: {gold_data['observation_month'].min()} to {gold_data['observation_month'].max()}")
    
    print(f"\n📋 Gold Layer Sample Data:")
    print(gold_data.head(10))
    
    print(f"\n📊 Data Quality Check:")
    print(f"   Missing Prime Rate: {gold_data['prime_rate'].isna().sum()}")
    print(f"   Missing CPI: {gold_data['headline_cpi'].isna().sum()}")
    print(f"   Missing ZAR/USD: {gold_data['zar_usd_exchange_rate'].isna().sum()}")
    
except Exception as e:
    print(f"❌ Gold layer processing failed: {str(e)}")
    # Use sample data for demonstration
    gold_manager = GoldLayerManager(None, PROJECT_ID, DATASET_ID)
    gold_data = gold_manager._create_sample_gold_data()
    print("🔧 Using sample gold data for demonstration")

## 6. Data Visualization and Analysis

### Core Analytical Question: 
**"How have changes in South Africa's prime interest rate and inflation rate historically correlated with the ZAR/USD exchange rate?"**

This section provides comprehensive visualizations and statistical analysis to answer this question definitively.

In [ ]:
# Prepare data for analysis
df_analysis = gold_data.copy()
df_analysis['observation_month'] = pd.to_datetime(df_analysis['observation_month'])

# Remove any rows with all NaN values
df_clean = df_analysis.dropna(subset=['prime_rate', 'headline_cpi', 'zar_usd_exchange_rate'], how='all')

print(f"📊 Analysis Dataset Summary:")
print(f"   Total observations: {len(df_clean)}")
print(f"   Date range: {df_clean['observation_month'].min().strftime('%Y-%m')} to {df_clean['observation_month'].max().strftime('%Y-%m')}")
print(f"   Complete data points: {df_clean.dropna().shape[0]}")

# Calculate basic statistics
print(f"\n📈 Descriptive Statistics:")
print(df_clean[['prime_rate', 'headline_cpi', 'zar_usd_exchange_rate']].describe().round(4))

In [ ]:
# Required Visualization 1: Time-Series Line Chart of All Three Indicators
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Prime Overdraft Rate (%)', 'Headline CPI (Index)', 'ZAR/USD Exchange Rate'),
    vertical_spacing=0.1,
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}],
           [{"secondary_y": False}]]
)

# Prime Rate
fig.add_trace(
    go.Scatter(
        x=df_clean['observation_month'],
        y=df_clean['prime_rate'],
        mode='lines',
        name='Prime Rate (%)',
        line=dict(color='#1f77b4', width=2),
        hovertemplate='<b>Prime Rate</b><br>Date: %{x}<br>Rate: %{y:.2f}%<extra></extra>'
    ),
    row=1, col=1
)

# CPI
fig.add_trace(
    go.Scatter(
        x=df_clean['observation_month'],
        y=df_clean['headline_cpi'],
        mode='lines',
        name='Headline CPI',
        line=dict(color='#ff7f0e', width=2),
        hovertemplate='<b>Headline CPI</b><br>Date: %{x}<br>Index: %{y:.2f}<extra></extra>'
    ),
    row=2, col=1
)

# ZAR/USD Exchange Rate
fig.add_trace(
    go.Scatter(
        x=df_clean['observation_month'],
        y=df_clean['zar_usd_exchange_rate'],
        mode='lines',
        name='ZAR/USD Rate',
        line=dict(color='#2ca02c', width=2),
        hovertemplate='<b>ZAR/USD Exchange Rate</b><br>Date: %{x}<br>Rate: %{y:.2f}<extra></extra>'
    ),
    row=3, col=1
)

# Update layout
fig.update_layout(
    title=dict(
        text='<b>South African Macroeconomic Indicators (2010-2024)</b><br><sub>Historical Trends in Prime Rate, Inflation, and Exchange Rate</sub>',
        x=0.5,
        font=dict(size=16)
    ),
    height=800,
    showlegend=False,
    font=dict(size=12)
)

# Update y-axes
fig.update_yaxes(title_text="Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Index (2016=100)", row=2, col=1)
fig.update_yaxes(title_text="ZAR per USD", row=3, col=1)

# Update x-axes
fig.update_xaxes(title_text="Year", row=3, col=1)

# Add gridlines for better readability
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)')

fig.show()

print("📊 Time-series visualization complete!")

In [ ]:
# Statistical Correlation Analysis
print("🔍 Statistical Correlation Analysis")
print("=" * 50)

# Prepare data for correlation analysis (remove NaN values)
correlation_data = df_clean[['prime_rate', 'headline_cpi', 'zar_usd_exchange_rate']].dropna()

# Calculate Pearson correlation coefficients
correlations = correlation_data.corr()
print("\n📊 Pearson Correlation Matrix:")
print(correlations.round(4))

# Calculate statistical significance
def calculate_correlation_with_pvalue(x, y):
    """Calculate correlation coefficient with p-value"""
    valid_data = pd.DataFrame({'x': x, 'y': y}).dropna()
    if len(valid_data) < 3:
        return np.nan, np.nan
    corr, p_value = pearsonr(valid_data['x'], valid_data['y'])
    return corr, p_value

print("\n📈 Detailed Correlation Analysis:")
print("-" * 40)

# Prime Rate vs ZAR/USD
pr_zar_corr, pr_zar_p = calculate_correlation_with_pvalue(
    correlation_data['prime_rate'], 
    correlation_data['zar_usd_exchange_rate']
)
print(f"Prime Rate ↔ ZAR/USD:")
print(f"   Correlation: {pr_zar_corr:.4f}")
print(f"   P-value: {pr_zar_p:.6f}")
print(f"   Significance: {'Significant' if pr_zar_p < 0.05 else 'Not significant'} (α = 0.05)")

# CPI vs ZAR/USD  
cpi_zar_corr, cpi_zar_p = calculate_correlation_with_pvalue(
    correlation_data['headline_cpi'], 
    correlation_data['zar_usd_exchange_rate']
)
print(f"\nCPI ↔ ZAR/USD:")
print(f"   Correlation: {cpi_zar_corr:.4f}")
print(f"   P-value: {cpi_zar_p:.6f}")
print(f"   Significance: {'Significant' if cpi_zar_p < 0.05 else 'Not significant'} (α = 0.05)")

# Prime Rate vs CPI
pr_cpi_corr, pr_cpi_p = calculate_correlation_with_pvalue(
    correlation_data['prime_rate'], 
    correlation_data['headline_cpi']
)
print(f"\nPrime Rate ↔ CPI:")
print(f"   Correlation: {pr_cpi_corr:.4f}")
print(f"   P-value: {pr_cpi_p:.6f}")
print(f"   Significance: {'Significant' if pr_cpi_p < 0.05 else 'Not significant'} (α = 0.05)")

# Interpret correlation strength
def interpret_correlation(corr):
    """Interpret correlation strength"""
    abs_corr = abs(corr)
    if abs_corr >= 0.8:
        return "Very Strong"
    elif abs_corr >= 0.6:
        return "Strong"
    elif abs_corr >= 0.4:
        return "Moderate"
    elif abs_corr >= 0.2:
        return "Weak"
    else:
        return "Very Weak"

print(f"\n🎯 Correlation Strength Interpretation:")
print(f"   Prime Rate ↔ ZAR/USD: {interpret_correlation(pr_zar_corr)} ({'Positive' if pr_zar_corr > 0 else 'Negative'})")
print(f"   CPI ↔ ZAR/USD: {interpret_correlation(cpi_zar_corr)} ({'Positive' if cpi_zar_corr > 0 else 'Negative'})")
print(f"   Prime Rate ↔ CPI: {interpret_correlation(pr_cpi_corr)} ({'Positive' if pr_cpi_corr > 0 else 'Negative'})")

In [ ]:
# Required Visualization 2: Correlation Matrix Heatmap and Scatter Plots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Correlation Matrix Heatmap',
        'Prime Rate vs ZAR/USD Exchange Rate',
        'CPI vs ZAR/USD Exchange Rate', 
        'Prime Rate vs CPI'
    ],
    specs=[[{"type": "heatmap"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "scatter"}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Correlation Matrix Heatmap
correlation_matrix = correlations.values
correlation_labels = correlations.columns.tolist()

fig.add_trace(
    go.Heatmap(
        z=correlation_matrix,
        x=['Prime Rate', 'CPI', 'ZAR/USD'],
        y=['Prime Rate', 'CPI', 'ZAR/USD'],
        colorscale='RdBu',
        zmid=0,
        text=correlation_matrix.round(3),
        texttemplate='%{text}',
        textfont={"size": 12},
        colorbar=dict(title="Correlation", x=0.47)
    ),
    row=1, col=1
)

# 2. Prime Rate vs ZAR/USD Scatter Plot
fig.add_trace(
    go.Scatter(
        x=correlation_data['prime_rate'],
        y=correlation_data['zar_usd_exchange_rate'],
        mode='markers',
        name='Prime Rate vs ZAR/USD',
        marker=dict(
            color='#1f77b4',
            size=6,
            opacity=0.7
        ),
        hovertemplate='<b>Prime Rate vs ZAR/USD</b><br>Prime Rate: %{x:.2f}%<br>ZAR/USD: %{y:.2f}<extra></extra>'
    ),
    row=1, col=2
)

# Add trendline for Prime Rate vs ZAR/USD
z = np.polyfit(correlation_data['prime_rate'].dropna(), 
               correlation_data['zar_usd_exchange_rate'].dropna(), 1)
p = np.poly1d(z)
x_trend = np.linspace(correlation_data['prime_rate'].min(), correlation_data['prime_rate'].max(), 100)
fig.add_trace(
    go.Scatter(
        x=x_trend,
        y=p(x_trend),
        mode='lines',
        name='Trend',
        line=dict(color='red', width=2, dash='dash'),
        showlegend=False
    ),
    row=1, col=2
)

# 3. CPI vs ZAR/USD Scatter Plot
fig.add_trace(
    go.Scatter(
        x=correlation_data['headline_cpi'],
        y=correlation_data['zar_usd_exchange_rate'],
        mode='markers',
        name='CPI vs ZAR/USD',
        marker=dict(
            color='#ff7f0e',
            size=6,
            opacity=0.7
        ),
        hovertemplate='<b>CPI vs ZAR/USD</b><br>CPI: %{x:.2f}<br>ZAR/USD: %{y:.2f}<extra></extra>'
    ),
    row=2, col=1
)

# Add trendline for CPI vs ZAR/USD
z2 = np.polyfit(correlation_data['headline_cpi'].dropna(), 
                correlation_data['zar_usd_exchange_rate'].dropna(), 1)
p2 = np.poly1d(z2)
x_trend2 = np.linspace(correlation_data['headline_cpi'].min(), correlation_data['headline_cpi'].max(), 100)
fig.add_trace(
    go.Scatter(
        x=x_trend2,
        y=p2(x_trend2),
        mode='lines',
        name='Trend',
        line=dict(color='red', width=2, dash='dash'),
        showlegend=False
    ),
    row=2, col=1
)

# 4. Prime Rate vs CPI Scatter Plot
fig.add_trace(
    go.Scatter(
        x=correlation_data['prime_rate'],
        y=correlation_data['headline_cpi'],
        mode='markers',
        name='Prime Rate vs CPI',
        marker=dict(
            color='#2ca02c',
            size=6,
            opacity=0.7
        ),
        hovertemplate='<b>Prime Rate vs CPI</b><br>Prime Rate: %{x:.2f}%<br>CPI: %{y:.2f}<extra></extra>'
    ),
    row=2, col=2
)

# Add trendline for Prime Rate vs CPI
z3 = np.polyfit(correlation_data['prime_rate'].dropna(), 
                correlation_data['headline_cpi'].dropna(), 1)
p3 = np.poly1d(z3)
x_trend3 = np.linspace(correlation_data['prime_rate'].min(), correlation_data['prime_rate'].max(), 100)
fig.add_trace(
    go.Scatter(
        x=x_trend3,
        y=p3(x_trend3),
        mode='lines',
        name='Trend',
        line=dict(color='red', width=2, dash='dash'),
        showlegend=False
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title=dict(
        text='<b>Correlation Analysis: South African Macroeconomic Indicators</b><br><sub>Statistical Relationships Between Prime Rate, Inflation, and Exchange Rate</sub>',
        x=0.5,
        font=dict(size=16)
    ),
    height=700,
    showlegend=False,
    font=dict(size=10)
)

# Update axes labels
fig.update_xaxes(title_text="Prime Rate (%)", row=1, col=2)
fig.update_yaxes(title_text="ZAR/USD Rate", row=1, col=2)

fig.update_xaxes(title_text="Headline CPI", row=2, col=1)
fig.update_yaxes(title_text="ZAR/USD Rate", row=2, col=1)

fig.update_xaxes(title_text="Prime Rate (%)", row=2, col=2)
fig.update_yaxes(title_text="Headline CPI", row=2, col=2)

fig.show()

print("📊 Correlation analysis visualization complete!")

### 📋 Analytical Findings Summary

**Research Question:** How have changes in South Africa's prime interest rate and inflation rate historically correlated with the ZAR/USD exchange rate?

#### Key Findings:

1. **Prime Rate ↔ ZAR/USD Exchange Rate:**
   - Correlation coefficient indicates the strength and direction of relationship
   - Statistical significance demonstrates reliability of the relationship
   - Economic interpretation: Higher interest rates may strengthen/weaken the Rand depending on economic context

2. **Inflation (CPI) ↔ ZAR/USD Exchange Rate:**
   - Shows the relationship between domestic inflation and currency valuation
   - Higher inflation typically weakens currency purchasing power
   - Important for monetary policy decisions

3. **Prime Rate ↔ Inflation (CPI):**
   - Demonstrates the effectiveness of monetary policy
   - Central bank uses interest rates to control inflation
   - Key relationship for understanding SARB policy effectiveness

#### Economic Context:
- **2010-2015**: Post-financial crisis recovery period
- **2016-2018**: Political uncertainty and credit rating downgrades
- **2020-2022**: COVID-19 pandemic and global economic disruption
- **2022-2024**: Global inflation surge and monetary policy responses

#### Policy Implications:
The relationships identified help understand how the South African Reserve Bank's monetary policy decisions (interest rate changes) affect both domestic inflation and international competitiveness through exchange rate movements.